In [1]:
import pandas as pd
import numpy as np
import os
import boto3
import s3fs

import os
import dask
from dask.distributed import Client
import dask.dataframe as dd
import dask_ml
import joblib
from collections import Counter
#import texthero

In [2]:
print(dask.__version__)

2021.05.1


In [3]:
file_list = ['listings_Asheville.csv',
 'listings_Austin.csv',
 'listings_Boston.csv',
 'listings_Broward County.csv',
 'listings_Cambridge.csv',
 'listings_Chicago.csv',
 'listings_Clark County.csv',
 'listings_Colombus.csv',
 'listings_Dallas.csv',
 'listings_Denver.csv',
 'listings_Hawaii.csv',
 'listings_Jersey City.csv',
 'listings_Los Angeles.csv',
 'listings_New Orleans.csv',
 'listings_New York City.csv',
 'listings_Oakland.csv',
 'listings_Pacific Grove.csv',
 'listings_Portland.csv',
 'listings_Rhode Island.csv',
 'listings_Salem.csv',
 'listings_San Diego.csv',
 'listings_San Franciso.csv',
 'listings_San Mateo.csv',
 'listings_Santa Clara.csv',
 'listings_Santa Cruz.csv',
 'listings_Seattle.csv',
 'listings_Twin Cities.csv',
 'listings_Washington.csv']

In [4]:
file_list2 = ['listings_San Diego.csv',
 'listings_San Franciso.csv',
 'listings_San Mateo.csv',
 'listings_Santa Clara.csv',
 'listings_Washington.csv']

In [5]:
client = Client(n_workers=2)

In [6]:
df = pd.DataFrame()
ddf = dd.from_pandas(df, npartitions=4)

In [7]:
def remove_col(dataframe):
    return dataframe.drop(['id', 
                    'listing_url', 
                    'scrape_id', 
                    'last_scraped', 
                    'picture_url', 
                    'host_url', 
                    'host_name', 
                    'host_thumbnail_url', 
                    'host_picture_url', 
                    'calendar_last_scraped', 
                    'license', 
                    'calculated_host_listings_count', 
                    'calculated_host_listings_count_entire_homes', 
                    'calculated_host_listings_count_private_rooms', 
                    'calculated_host_listings_count_shared_rooms', 
                    'minimum_minimum_nights', 
                    'maximum_minimum_nights', 
                    'minimum_maximum_nights', 
                    'maximum_maximum_nights', 
                    'minimum_nights_avg_ntm', 
                    'maximum_nights_avg_ntm', 
                    'calendar_updated',
                    'neighbourhood_group_cleansed',
                    'bathrooms'], axis=1)

def words_to_dict(words, score):
    return dict.fromkeys(set(words), score)

def clean_text(text, val):
    text = str(text)
    text = text.lower()
    codes = ['<br />', '<b>', '</b>']
    for code in codes:
         text = text.replace(code, ' ')


    stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
    for word in stop_words:
        text = text.replace(' ' + word + ' ',' ')
    text = text.replace('/', ' ')
    text = text.replace("'", '')
    accept_string = 'abcdefghijklmnopqrstuvwxyz'
    new_str = ''
    for char in text:
        if char in accept_string:
            new_str = new_str + char
        else:
            new_str = new_str + ' '
    new_str = new_str.replace('   ', ' ').replace('  ', ' ').split(' ')
    out_dict = words_to_dict(new_str, val)
    return out_dict

def word_list(row_dict):
    out_list = []
    for key in row_dict:
        out_list.append(key)
    return out_list

def divide_dicts(dict1, dict2):
    delete_list = []
    for word in dict1:
        if dict2[word] > 100:
            dict1[word] = dict1[word] / dict2[word]
        else:
            delete_list.append(word)
    [dict1.pop(key) for key in delete_list]
    return dict1

def text_score(dict1, dict2):
    score = 0
    count = 0
    for k in dict1:
        if k in dict2.keys():
            score = score + dict2[k]
            count += 1
    return (score+1)/(count+1)

In [8]:
#Dask Way
df = pd.DataFrame()
ddf = dd.from_pandas(df, npartitions=10)
for file in file_list2:
    data_fname = 'C:\\Users\\harri\\Documents\\GitHub\\DSE230\\datasets\\' + file
    data = pd.read_csv(data_fname)
    
    d_data = dd.from_pandas(data, npartitions=1)
    ddf = ddf.append(d_data)
#ddf = dd.read_csv(data_fname, blocksize=None)

with joblib.parallel_backend("dask"):   
    #Clean Dataset
    ddf = remove_col(ddf)
    ddf = ddf.dropna(subset=['review_scores_value'])

    ddf = ddf.reset_index(drop=True)
    text_columns = ['host_about', 'description', 'neighborhood_overview', 'name']

    #Turn text to dictionaries of pertinent words with review score values
    for col in text_columns:
        ddf[col] = ddf.apply(lambda x: clean_text(x[col], x['review_scores_value']), axis=1)

    for col in text_columns:
        temp_ddf = ddf.apply(lambda x: word_list(x[col]), axis=1)
        full_word_list = dask.array.concatenate(temp_ddf, axis=0)
        word_counts = Counter(full_word_list.compute())

        word_scores = ddf[col].compute()
        dicts = list(word_scores)
        c = Counter()
        for d in dicts:
            c.update(d)
        weight_counts = divide_dicts(dict(c), dict(word_counts))

        ddf[col] = ddf.apply(lambda x: text_score(x[col], weight_counts), axis=1)

C:\Users\harri\anaconda3\lib\site-packages\dask\dataframe\core.py:4740: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))
C:\Users\harri\anaconda3\lib\site-packages\dask\dataframe\core.py:4740: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'float64'))

  warnings.warn(met

In [313]:
#Pandas Way
for file in file_list:
    data_fname = "s3://harrybucket6/DSE230 Project/" + file

    #Clean Dataset
    data = pd.read_csv(data_fname)
    data = remove_col(data)
    data = data.dropna(subset=['review_scores_value'])

    data.reset_index(inplace=True, drop=True)

    text_columns = ['host_about', 'description', 'neighborhood_overview', 'name']

    #Turn text to dictionaries of pertinent words with review score values
    for col in text_columns:
        data[col] = data.apply(lambda x: clean_text(x[col], x['review_scores_value']), axis=1)

    dict_weight_counts = {}
    words_df = pd.DataFrame()
    for col in text_columns:
        words_df[col] = data.apply(lambda x: word_list(x[col]), axis=1)
        word_counts = Counter(words_df[col].apply(pd.Series).stack())

        dicts = list(data[col])
        c = Counter()
        for d in dicts:
            c.update(d)
        dict_weight_counts[col] = divide_dicts(dict(c), dict(word_counts))

    for col in text_columns:
        data[col] = data.apply(lambda x: text_score(x[col], dict_weight_counts[col]), axis=1)
        
    with joblib.parallel_backend("dask"):    
            d_data = dd.from_pandas(data, npartitions=4)
            ddf = ddf.append(d_data)
            
ddf.reset_index(drop=True)

C:\Users\harri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (67) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,description,neighborhood_overview,host_id,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
npartitions=169,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,object,object,float64,float64,object,object,float64,object,float64,float64,object,object,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,object,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [116]:
ddf.shape

(Delayed('int-2d05cb38-e2b3-4438-ab9e-44edd5fe87fc'), 50)

In [336]:
client.shutdown()

In [422]:
data_fname = "s3://harrybucket6/DSE230 Project/" + 'listings_San Diego.csv'
df = pd.read_csv(data_fname)
df = remove_col(df)
df = df.dropna(subset=['review_scores_value'])

df.reset_index(inplace=True, drop=True)

text_columns = ['host_about', 'description', 'neighborhood_overview', 'name']

#Turn text to dictionaries of pertinent words with review score values
for col in text_columns:
    df[col] = df.apply(lambda x: clean_text(x[col], x['review_scores_value']), axis=1)



In [423]:
dict_weight_counts = {}
words_df = pd.DataFrame()
words_df['host_about'] = df.apply(lambda x: word_list(x['host_about']), axis=1)
word_counts = Counter(words_df['host_about'].apply(pd.Series).stack())

In [447]:
len(words_df['host_about'])

7750

In [435]:
len(words_df['host_about'].apply(pd.Series).stack())

214925

In [441]:
len(np.concatenate(words_df['host_about']))

214925

In [ ]:
dict_weight_counts = {}
words_df = pd.DataFrame()
for col in text_columns:
    words_df[col] = df.apply(lambda x: word_list(x[col]), axis=1)
    word_counts = Counter(words_df[col].apply(pd.Series).stack())
    
    dicts = list(df[col])
    c = Counter()
    for d in dicts:
        c.update(d)
    dict_weight_counts[col] = divide_dicts(dict(c), dict(word_counts))

for col in text_columns:
    df[col] = df.apply(lambda x: text_score(x[col], dict_weight_counts[col]), axis=1)

In [297]:
dict_weight_counts['host_about']

{'': 9.459734167318217,
 'life': 9.483775811209439,
 'nice': 9.459677419354838,
 'offer': 9.336363636363636,
 'home': 9.440182054616384,
 'back': 9.537906137184116,
 'town': 9.582089552238806,
 'enjoy': 9.58008658008658,
 'san': 9.500671591672264,
 'year': 9.545833333333333,
 'vacation': 9.223661485319516,
 'part': 9.431952662721894,
 'we': 9.440333024976873,
 'travel': 9.557364341085272,
 'lot': 9.579310344827586,
 'here': 9.532203389830508,
 'diego': 9.496002780674313,
 'outdoor': 9.421052631578947,
 'living': 9.466101694915254,
 'others': 9.527027027027026,
 'love': 9.567384875369667,
 'great': 9.572443181818182,
 'traveling': 9.505586592178771,
 'lifestyle': 9.594059405940595,
 'you': 9.458188153310104,
 'beautiful': 9.470684039087947,
 'look': 9.49877750611247,
 'contact': 9.283018867924529,
 'friends': 9.545454545454545,
 'fun': 9.621118012422361,
 'work': 9.506151142355009,
 'many': 9.526970954356846,
 'welcome': 9.48081841432225,
 'im': 9.53925353925354,
 'days': 9.452702702702

In [302]:
df['host_about'] = df.apply(lambda x: text_score(x['host_about'], dict_weight_counts['host_about']), axis=1)

In [303]:
test

0       9.153632
1       9.291806
2       9.253836
3       9.181984
4       8.305187
          ...   
7745    9.109375
7746    5.221898
7747    5.221898
7748    7.379526
7749    5.221898
Length: 7750, dtype: float64

In [305]:
df

,name,description,neighborhood_overview,host_id,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,8.011987,9.340263,5.125139,129123,2010-05-21,"San Diego, California, United States",9.153632,within an hour,100%,52%,...,2020-11-29,98.0,10.0,10.0,10.0,10.0,10.0,10.0,f,0.48
1,5.315534,9.370268,9.101516,164137,2010-07-12,"San Diego, California, United States",9.291806,within a day,100%,83%,...,2019-10-20,86.0,9.0,9.0,10.0,10.0,9.0,9.0,f,1.11
2,7.381763,9.347228,9.069395,302986,2010-11-28,"San Diego, California, United States",9.253836,within an hour,100%,96%,...,2021-02-28,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,5.05
3,7.801956,9.382144,9.115170,21532,2009-06-14,"San Diego, California, United States",9.181984,within an hour,100%,97%,...,2021-04-02,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,7.17
4,5.180000,9.362310,8.699817,332552,2011-01-02,"San Diego, California, United States",8.305187,within a day,100%,43%,...,2021-03-02,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,7.838013,9.356682,9.127546,22459966,2014-10-12,"San Diego, California, United States",9.109375,within an hour,100%,70%,...,2021-03-14,100.0,10.0,8.0,10.0,10.0,8.0,10.0,f,1.00
7746,5.215136,9.348574,8.131114,53501307,2016-01-05,"San Diego, California, United States",5.221898,within an hour,100%,100%,...,2021-03-14,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,3.00
7747,7.761829,9.344202,9.042790,31991928,NaN,NaN,5.221898,NaN,NaN,NaN,...,2021-03-15,80.0,10.0,8.0,8.0,8.0,10.0,10.0,t,1.00
7748,7.331713,9.244327,5.125139,41582938,2015-08-15,"San Diego, California, United States",7.379526,within an hour,100%,100%,...,2021-03-16,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,1.00


In [281]:
z = ['a','b']
test_dict = {'a':0, 'b':1, 'c':2}
[test_dict.pop(key) for key in z]

[0, 1]

In [282]:
test_dict

{'c': 2}

In [413]:
t1 = pd.DataFrame()
t1['first'] = [[2,3,4],[6,1,5]]
#np.concatenate(t1)

In [444]:
np.concatenate(t1['first'], axis = 0)

array([2, 3, 4, 6, 1, 5])